# Train/dev split

In [1]:
from sklearn.model_selection import train_test_split

In [2]:
texts, ids = [], []
with open('train_reviews.txt') as f:
    for line in f:
        text_id, text = line.rstrip('\r\n').split('\t')
        texts.append(text)
        ids.append(text_id)

In [3]:
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [4]:
train_aspects, dev_aspects = [], []
with open('train_aspects.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_aspects.append(line)
        if text_id in dev_ids:
            dev_aspects.append(line)

In [5]:
train_sentiment, dev_sentiment = [], []
with open('train_cats.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_sentiment.append(line)
        if text_id in dev_ids:
            dev_sentiment.append(line)

In [6]:
with open('train_split_aspects.txt', 'w') as f:
    for l in train_aspects:
        print(l, file=f)
with open('dev_aspects.txt', 'w') as f:
    for l in dev_aspects:
        print(l, file=f)
with open('train_split_reviews.txt', 'w') as f:
    for i, l in zip(train_ids, train_texts):
        print(i, l, sep="\t", file=f)
with open('dev_reviews.txt', 'w') as f:
    for i, l in zip(dev_ids, dev_texts):
        print(i, l, sep="\t", file=f)
with open('train_split_cats.txt', 'w') as f:
    for l in train_sentiment:
        print(l, file=f)
with open('dev_cats.txt', 'w') as f:
    for l in dev_sentiment:
        print(l, file=f)

# Baseline 1,2: категория и тональность упоминаний

Выделяем только аспекты, встретившиеся в train'е, приписываем самую частотную категорию.

In [1]:
import pandas as pd

In [2]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])

In [3]:
import stanza
stanza.download('ru')

2021-12-25 20:29:40 INFO: Downloading default packages for language: ru (Russian)...
2021-12-25 20:29:42 INFO: File exists: /Users/rhubarb/stanza_resources/ru/default.zip.
2021-12-25 20:29:48 INFO: Finished downloading models and saved to /Users/rhubarb/stanza_resources.


In [4]:
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

2021-12-25 20:29:48 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| lemma     | syntagrus |

2021-12-25 20:29:48 INFO: Use device: cpu
2021-12-25 20:29:48 INFO: Loading: tokenize
2021-12-25 20:29:48 INFO: Loading: lemma
2021-12-25 20:29:48 INFO: Done loading processors!


In [5]:
def normalize(text):
    doc = nlp(text)
    words = [word.lemma for sent in doc.sentences for word in sent.words]
    return words

In [6]:
from collections import defaultdict, Counter

In [7]:
train_asp['norm_mention'] = [tuple(normalize(m)) for m in train_asp['mention']]

Строим частотный словарь "токенизированное упоминание + категория"

Категория - аспектная категория или тональность

In [8]:
def get_mention_category(data, cat_type):
    mention_categories = data.value_counts(subset=['norm_mention', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
    return {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items()}

In [9]:
best_mention_cat = get_mention_category(train_asp, 'category')

In [10]:
best_mention_cat

{('ресторан',): 'Whole',
 ('интерьер',): 'Interior',
 ('обслуживание',): 'Service',
 ('официант',): 'Service',
 ('заведение',): 'Whole',
 ('кухня',): 'Food',
 ('место',): 'Whole',
 ('блюдо',): 'Food',
 ('еда',): 'Food',
 ('цена',): 'Price',
 ('персонал',): 'Service',
 ('пиво',): 'Food',
 ('порция',): 'Food',
 ('официантка',): 'Service',
 ('атмосфера',): 'Interior',
 ('меню',): 'Food',
 ('музыка',): 'Interior',
 ('сервис',): 'Service',
 ('встретить',): 'Service',
 ('девушка',): 'Service',
 ('горячий',): 'Food',
 ('десерт',): 'Food',
 ('администратор',): 'Service',
 ('обстановка',): 'Interior',
 ('вечер',): 'Whole',
 ('салаты',): 'Food',
 ('зал',): 'Interior',
 ('мясо',): 'Food',
 ('ждать',): 'Service',
 ('обслуживал',): 'Service',
 ('салат',): 'Food',
 ('поесть',): 'Food',
 ('столик',): 'Interior',
 ('вино',): 'Food',
 ('кафе',): 'Whole',
 ('чай',): 'Food',
 ('принести',): 'Service',
 ('вкус',): 'Food',
 ('напиток',): 'Food',
 ('молодой', 'человек'): 'Service',
 ('шашлык',): 'Food',
 ('

In [11]:
best_mention_sentiment = get_mention_category(train_asp, 'sentiment')

In [12]:
best_mention_sentiment

{('интерьер',): 'positive',
 ('обслуживание',): 'positive',
 ('ресторан',): 'positive',
 ('место',): 'positive',
 ('кухня',): 'positive',
 ('официант',): 'positive',
 ('заведение',): 'positive',
 ('персонал',): 'positive',
 ('еда',): 'positive',
 ('цена',): 'positive',
 ('блюдо',): 'positive',
 ('атмосфера',): 'positive',
 ('порция',): 'positive',
 ('музыка',): 'positive',
 ('пиво',): 'positive',
 ('меню',): 'positive',
 ('встретить',): 'positive',
 ('обстановка',): 'positive',
 ('официантка',): 'positive',
 ('вечер',): 'positive',
 ('девушка',): 'positive',
 ('администратор',): 'positive',
 ('зал',): 'positive',
 ('горячий',): 'neutral',
 ('обслуживал',): 'positive',
 ('сервис',): 'positive',
 ('десерт',): 'positive',
 ('ждать',): 'negative',
 ('салаты',): 'positive',
 ('поесть',): 'positive',
 ('столик',): 'positive',
 ('живой', 'музыка'): 'positive',
 ('напиток',): 'neutral',
 ('молодой', 'человек'): 'positive',
 ('провести', 'время'): 'positive',
 ('вкус',): 'positive',
 ('закуска'

In [13]:
dev_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id', 'text'])

In [14]:
dev_texts

,text_id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...
...,...,...
66,9216,Вы брали этот ресторан так как он близко от до...
67,8996,"Были с друзьями в пабе Метрополь, всё очень по..."
68,38299,"Случайно увидели акцию на сайте купонов, решил..."
69,37819,Очень долго выбирали ресторан на Новогодний ка...


Длины упоминаний аспектов в трейне:

In [16]:
Counter([len(x) for x in best_mention_sentiment.keys()])

Counter({1: 591, 2: 321, 3: 158, 4: 61, 5: 24, 6: 12, 7: 5, 9: 2, 10: 1, 8: 1})

Будем учитывать только упоминания длиной 1-5 токенов:

In [17]:
def label_texts(text, mentions, sentiments, max_len=5):
    tokenized = [word for sent in nlp(text).sentences for word in sent.words]
    text_end = len(tokenized)
    for i, token in enumerate(tokenized):
        for l in reversed(range(max_len)):
            if i + l > text_end:
                continue
            span = tokenized[i:i + l]
            key = tuple([t.lemma for t in span])
            if key in mentions:
                start, end = span[0].start_char, span[-1].end_char
                yield mentions[key], text[start:end], start, end, sentiments[key]
                break

Применяем частотные данные к текстам из dev:

In [18]:
with open('dev_pred_aspects.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for asp in label_texts(text, best_mention_cat, best_mention_sentiment):
            print(idx, *asp, sep="\t", file=f)

# Baseline 3

Посчитаем упоминания аспектов с предсказанной тональностью, припишем
- `absence` - если нет упоминаний данной категории
- `both` - если есть упоминания с разной тональностью
- `positive/neutral/negative` - если все упоминания одной тональности

In [19]:
CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']

In [20]:
def get_full_sentiment(text, mentions, sentiment, max_len=5):
    asp_counter = defaultdict(Counter)
    for asp in label_texts(text, best_mention_cat, best_mention_sentiment, max_len):
        category, *_, sentiment = asp
        asp_counter[category][sentiment] += 1
    for c in CATEGORIES:
        if not asp_counter[c]:
            s = 'absence'
        elif len(asp_counter[c]) == 1:
            s = asp_counter[c].most_common(1)[0][0]
        else:
            s = 'both'
        yield c, s

In [21]:
with open('dev_pred_cats.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for c, s in get_full_sentiment(text, best_mention_cat, best_mention_sentiment):
            print(idx, c, s, sep="\t", file=f)